In [1]:
import pandas as pd

In [138]:
from tqdm import tqdm

tqdm.pandas()

In [49]:
TOPICOS = [
    "amnistia", #ok
    "feminismo", #ok
    "genero", #ok
    "monarquia", #ok
    "salud", #ok
    "territorio", #ok
    "violencia_genero", 
    "vivienda", 
    "educacion", 
    "economia", 
    "inmigracion", 
]

In [90]:
topic = TOPICOS[3]
print("Topico elegido", topic)


Topico elegido amnistia


In [115]:
dfs = []
for topic in TOPICOS: 
    df= pd.read_csv(f'comentarios_{topic}_etiquetado.csv') 
    df['topico'] = topic 
    dfs.append(df) 
df_total = pd.concat(dfs)

In [116]:
df_total.shape

(48973, 15)

In [130]:
df_total.head().T

,0,1,2,3,4
video_id,T-smnnEAdM0,T-smnnEAdM0,T-smnnEAdM0,T-smnnEAdM0,T-smnnEAdM0
author,UCTlJjNviHj7Y32eiOdHH_FQ,UC6o_ckmEk87f7TiViatW11w,UCCXQ_BAzI-GcXgDUwN4XiXQ,UCB4dg7xkDRLpW561xlwNnyw,UCFc7zfxN235QsmGBJ-GsUcA
comment,No me creo que les importemos algo. Estos son ...,No a la ANMISTIA y no AL PERDONA LA DEUDA DE C...,"Que no te enteras,Contreras\nQue se la trae fl...","¿Como puede ser impecable?, si hasta hace poco...",Osea que cuando gobierna el PP nos roba y mete...
comment_stem,creo importemo misma opini,anmistia perdona deuda catalu sirv separatista...,entera contrera trae floja asunto interno espa...,pued impec hace bocachancla palanganero felon ...,osea gobierna roba mete guerra aqui pasa gobie...
vectores_tfidf,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.]
sentiment,0.0,-1.0,-1.0,-1.0,-1.0
sentiment_2,-1.0,-1.0,-1.0,-1.0,-1.0
hateful,0.073077,0.05202,0.562841,0.115163,0.645626
targeted,0.004245,0.005793,0.014446,0.025937,0.008227
aggressive,0.030084,0.051741,0.309607,0.126386,0.292196


In [131]:
df_total.topico.value_counts()

amnistia            8305
feminismo           6254
inmigracion         5396
violencia_genero    4925
territorio          4484
monarquia           4190
vivienda            4016
genero              3958
economia            3345
salud               2323
educacion           1777
Name: topico, dtype: int64

In [132]:
df_total.to_csv('TODOS_comentarios_negativos_a_clasificar')

In [133]:
import pickle
# Cargar el modelo guardado
with open("SVC_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [135]:
df_total['comment'] = df_total['comment'].apply(lambda x: [x])

In [137]:
df_total['comment']

0       [No me creo que les importemos algo. Estos son...
1       [No a la ANMISTIA y no AL PERDONA LA DEUDA DE ...
2       [Que no te enteras,Contreras\nQue se la trae f...
3       [¿Como puede ser impecable?, si hasta hace poc...
4       [Osea que cuando gobierna el PP nos roba y met...
                              ...                        
5391    [Para acumular votos para las elecciones presi...
5392                                [Por favor no voten.]
5393    [NUESTROS HIJOS, NIETOS y BISNIETOS, NUNCA LE ...
5394    [Pueden sacar 15 o 20 millones de SI, acaso co...
5395    [El referendun no es mas que un fraude elector...
Name: comment, Length: 48973, dtype: object

In [139]:
def asignar_categoria(elemento):
    if isinstance(elemento, list):
        resultado = loaded_model.predict(elemento) 
        categoria_predicha = resultado[0]
        
        return categoria_predicha
    else:
        raise ValueError(f"{elemento} no es una lista")

# Aplicar la función a la columna 'comment' y crear una nueva columna 'category'
df_total['category'] = df_total['comment'].progress_apply(lambda x: asignar_categoria(x))

100%|██████████████████████████████████████████████████████████████████████████| 48973/48973 [00:45<00:00, 1078.24it/s]


In [140]:
df_total['category'].value_counts()

ideología política    33113
machismo               9240
racismo                6620
Name: category, dtype: int64

In [147]:
loaded_model.predict_proba(df_total['category'])

array([[0.29223723, 0.44022138, 0.26754138],
       [0.3373842 , 0.47464574, 0.18797005],
       [0.3373842 , 0.47464574, 0.18797005],
       ...,
       [0.3373842 , 0.47464574, 0.18797005],
       [0.3373842 , 0.47464574, 0.18797005],
       [0.3143202 , 0.5097721 , 0.1759077 ]])

In [148]:
df_total.to_csv('TODOS_comentarios_negativos_calificados')